In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import panel as pn
from panel_vega_view import VegaView

pn.extension('ace', 'jsoneditor', 'vega')


spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Drag out a rectangular brush to highlight points.",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/refs/heads/main/docs/data/cars.json"},
  "params": [{
    "name": "brush",
    "select": "interval",
    "value": {"x": [55, 160], "y": [13, 37]}
  },{
    "name": "point_sel",
    "select": "point",
  },
  ],
  "mark": "point",
  "encoding": {
    "x": {"field": "Horsepower", "type": "quantitative"},
    "y": {"field": "Miles_per_Gallon", "type": "quantitative"},
    "color": {
      "condition": {"param": "brush", "field": "Cylinders", "type": "ordinal"},
      "value": "grey"
    },
    "size": {
      "condition": {"param": "point_sel", "value": 100},
      "value": 10,
    },
    "facet": {
      "field": "Cylinders",
      "columns": 2
    }
  }
}


spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/movies.json"},
  "transform": [{
    "filter": {"and": [
      {"field": "IMDB Rating", "valid": True},
      {"field": "Rotten Tomatoes Rating", "valid": True}
    ]}
  }],
  "params": [
      {
        "name": "colorDomain",
        "value": [0, 30],
      },
      {
        "name": "brush",
        "select": "interval",
        # "value": {"x": [55, 160], "y": [13, 37]}
      },
      {
          "name": "x_bins",
          "value": 40,
      },
      {
          "name": "y_bins",
          "value": 40,
      },
      {
        "name": "point_sel",
        "select": "point",
      },
  ],
  "mark": {
      "type": "rect",
      # "width": 5,
      # "height": 5,
  },
  # "width": "container",
  "height": {"step": 10},
  "width": {"step": 10},
  "encoding": {
    # "size": 5,
    "x": {
      "bin": {"maxbins": 40},
      "field": "IMDB Rating",
      "type": "ordinal"
    },
    "y": {
      "bin": {"maxbins": 40},
      "field": "Rotten Tomatoes Rating",
      "type": "ordinal"
    },
    "color": {
      "condition": {
        "param": "brush",
        "aggregate": "count",
        "type": "quantitative",
        "scale": {
            "domain": {"expr": "colorDomain"},
        }
      },
      "value": "grey"
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
  }
    }
}


spec = {
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A contour plot example, overlaying a density estimate on scatter plot points.",
  "width": 500,
  "height": 400,
  "padding": 5,
  "autosize": "pad",
  "background": "white",

  "signals": [
    {
      "name": "bandwidth", "value": -1,
      "bind": {"input": "range", "min": -1, "max": 100, "step": 1}
    },
    {
      "name": "resolve", "value": "shared",
      "bind": {"input": "select", "options": ["independent", "shared"]}
    },
    {
      "name": "counts", "value": True,
      "bind": {"input": "checkbox"}
    }
  ],

  "data": [
    {
      "name": "source",
      "url": "https://raw.githubusercontent.com/vega/vega/refs/heads/main/docs/data/cars.json",
      "transform": [
        {
          "type": "filter",
          "expr": "datum.Horsepower != null && datum.Miles_per_Gallon != null"
        }
      ]
    },
    {
      "name": "density",
      "source": "source",
      "transform": [
        {
          "type": "kde2d",
          "groupby": ["Origin"],
          "size": [{"signal": "width"}, {"signal": "height"}],
          "x": {"expr": "scale('x', datum.Horsepower)"},
          "y": {"expr": "scale('y', datum.Miles_per_Gallon)"},
          "bandwidth": {"signal": "[bandwidth, bandwidth]"},
          "counts": {"signal": "counts"}
        }
      ]
    },
    {
      "name": "contours",
      "source": "density",
      "transform": [
        {
          "type": "isocontour",
          "field": "grid",
          "resolve": {"signal": "resolve"},
          "levels": 3
        }
      ]
    }
  ],

  "scales": [
    {
      "name": "x",
      "type": "linear",
      "round": True,
      "nice": True,
      "zero": True,
      "domain": {"data": "source", "field": "Horsepower"},
      "range": "width"
    },
    {
      "name": "y",
      "type": "linear",
      "round": True,
      "nice": True,
      "zero": True,
      "domain": {"data": "source", "field": "Miles_per_Gallon"},
      "range": "height"
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": {
        "data": "source", "field": "Origin",
        "sort": {"order": "descending"}
      },
      "range": "category"
    }
  ],

  "axes": [
    {
      "scale": "x",
      "grid": True,
      "domain": False,
      "orient": "bottom",
      "tickCount": 5,
      "title": "Horsepower"
    },
    {
      "scale": "y",
      "grid": True,
      "domain": False,
      "orient": "left",
      "titlePadding": 5,
      "title": "Miles_per_Gallon"
    }
  ],

  "legends": [
    {"stroke": "color", "symbolType": "stroke"}
  ],

  "marks": [
    {
      "name": "marks",
      "type": "symbol",
      "from": {"data": "source"},
      "encode": {
        "update": {
          "x": {"scale": "x", "field": "Horsepower"},
          "y": {"scale": "y", "field": "Miles_per_Gallon"},
          "size": {"value": 4},
          "fill": {"value": "#ccc"}
        }
      }
    },
    {
      "type": "image",
      "from": {"data": "density"},
      "encode": {
        "update": {
          "x": {"value": 0},
          "y": {"value": 0},
          "width": {"signal": "width"},
          "height": {"signal": "height"},
          "aspect": {"value": False}
        }
      },
      "transform": [
        {
          "type": "heatmap",
          "field": "datum.grid",
          "resolve": {"signal": "resolve"},
          "color": {"expr": "scale('color', datum.datum.Origin)"}
        }
      ]
    },
    {
      "type": "path",
      "clip": True,
      "from": {"data": "contours"},
      "encode": {
        "enter": {
          "strokeWidth": {"value": 1},
          "strokeOpacity": {"value": 1},
          "stroke": {"scale": "color", "field": "Origin"}
        }
      },
      "transform": [
        { "type": "geopath", "field": "datum.contour" }
      ]
    }
  ]
}



vv = VegaView(
    spec=spec,
    opt={
        "actions": True,
    },
    signal_names=[
        # "point_sel",
        # "brush",
        # "colorDomain",
        # "height",
        # "brush_x",
        # "brush_tuple",
    ],
)

json_editor = pn.widgets.JSONEditor(value=vv.param.signals, width=300)
json_editor.jslink(vv, code={"value": """
console.log('JSON', source);
console.log('JSON', target.model_proxy.uuid);
let vegaView = window.vegaViews[target.model_proxy.uuid];

Object.entries(source.value).forEach(([key, value]) => {
  console.log(key, value);
  if (value != null && value != undefined)
    vegaView.signal(key, value);
});
vegaView.runAsync();
"""})
# json_editor2 = pn.widgets.JSONEditor(value=vv.param.data, width=300)
color_lims = pn.widgets.RangeSlider(name='Color limits', start=0, end=125, value=(0, 40), step=1)
color_lims.jslink(vv, code={'value': """
let vegaView = window.vegaViews[target.model_proxy.uuid];

vegaView.signal("colorDomain", source.value);
vegaView.runAsync();
"""})

width_slider = pn.widgets.IntSlider(name='Width', start=100, end=1000, value=300)
width_slider.jslink(vv, code={'value': """
              
let signals = {...target.model_proxy.signals};
signals["width"] = source.value;
target.model_proxy.signals = signals;
"""})

maxbins_slider = pn.widgets.IntSlider(name='MaxBins', start=1, end=100, value=5)
maxbins_slider.jslink(vv, code={'value': """       
let spec = {...target.model_proxy.spec};
console.log('SPEC', spec);
spec.encoding.x.bin.maxbins = source.value;
spec.encoding.y.bin.maxbins = source.value;
target.model_proxy.spec = spec;
"""})

uuid = pn.widgets.TextInput(value=vv.param.uuid)

pn.Column(
  pn.Row(
      vv,
      json_editor,
      # json_editor2,
  ),
  color_lims,
  # width_slider,
  maxbins_slider,
  uuid,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8f16c062-e98e-43d1-af06-87e54c610749': {'version…

In [32]:
vv.data_names

['root',
 'brush_store',
 'point_sel_store',
 'source_0',
 'facet_domain',
 'facet_domain_row',
 'facet_domain_column',
 'facet-title',
 'row_header',
 'column_footer',
 'cell']

In [37]:
vv.data_name = 'source_0'


In [38]:
vv.vega_data

[]

In [31]:
vv.spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Drag out a rectangular brush to highlight points.",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/refs/heads/main/docs/data/cars.json"},
  "params": [{
    "name": "brush",
    "select": "interval",
    "value": {"x": [55, 160], "y": [13, 37]}
  },{
    "name": "point_sel",
    "select": "point",
  },
  ],
  "mark": "point",
  "encoding": {
    "x": {"field": "Horsepower", "type": "quantitative"},
    "y": {"field": "Miles_per_Gallon", "type": "quantitative"},
    "color": {
      "condition": {"param": "brush", "field": "Cylinders", "type": "ordinal"},
      "value": "grey"
    },
    "size": {
      "condition": {"param": "point_sel", "value": 100},
      "value": 10,
    },
    "facet": {
      "field": "Cylinders",
      "columns": 2
    }
  }
}



In [12]:
from copy import deepcopy

new_spec = deepcopy(vv.spec)
new_spec['mark'] = 'rect'
new_spec['encoding']['x']['bin']['maxbins'] = 20
new_spec['encoding']['y']['bin']['maxbins'] = 20
vv.spec = new_spec

In [2]:
vv.transformed_data

[{'contour': {'type': 'MultiPolygon',
   'value': 0.0033975893165916204,
   'coordinates': [[[[371.8277256334293, 318],
      [374, 317.5965366295171],
      [378, 316.7333911709155],
      [382, 315.7204755442789],
      [386, 314.57804773329013],
      [387.9084309035209, 314],
      [390, 313.18780573922123],
      [394, 311.44325285343973],
      [396.84045049460656, 310],
      [398, 309.2149564821127],
      [402, 306.4960150650607],
      [402.7637012942774, 306],
      [406, 302.35475368293316],
      [406.32477931481804, 302],
      [408.20638375889786, 298],
      [407.85071821325624, 294],
      [406, 290.1289289154737],
      [405.94971581352473, 290],
      [402.75256126761485, 286],
      [402, 285.32759054056424],
      [398.837458738999, 282],
      [398, 281.3511777132958],
      [394.0975203891459, 278],
      [394, 277.9266071887737],
      [390, 274.8326892805969],
      [388.9548772390587, 274],
      [386, 272.016658619264],
      [382.9693593314763, 270],
      [